<a href="https://colab.research.google.com/github/lucius-yu/colab-tutorial/blob/master/kfold_average_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KFlod Average on DNN for fashion mnist problem

Import liberary

In [0]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score

Load the data, preprocess the data, setup KFold CV

In [4]:
mnist = tf.keras.datasets.fashion_mnist

# load data
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# normalization
training_images  = training_images / 255.0
test_images = test_images / 255.0

4423680/4422102 [==============================] - 0s 0us/step


Build the model

In [0]:
def get_model():
  # build the model
  model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                      tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                      tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                      tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
  model.compile(optimizer = tf.train.AdamOptimizer(learning_rate=0.0005),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

Train KFold Model with early stopping

In [27]:
# setup cv
skf = StratifiedKFold(n_splits=5)

# variables
num_epochs=20

# predictions
test_preds = list()

for train_index, validation_index in skf.split(training_images, training_labels):
  print("TRAIN:", len(train_index), "Validation:", len(validation_index))
  
  X_train, X_valid = training_images[train_index], training_images[validation_index]
  y_train, y_valid = training_labels[train_index], training_labels[validation_index]
  
  dnn_model = get_model()
  dnn_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=num_epochs)
  
  # evaluate on test images
  dnn_model.evaluate(test_images, test_labels)
  
  # record test_preds
  test_preds.append(dnn_model.predict(test_images, batch_size=128))

TRAIN: 48000 Validation: 12000
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 6s 134us/sample - loss: 0.5352 - acc: 0.8145 - val_loss: 0.4224 - val_acc: 0.8467
Epoch 2/20
48000/48000 [==============================] - 6s 127us/sample - loss: 0.3833 - acc: 0.8627 - val_loss: 0.3470 - val_acc: 0.8753
Epoch 3/20
48000/48000 [==============================] - 6s 125us/sample - loss: 0.3405 - acc: 0.8753 - val_loss: 0.3365 - val_acc: 0.8793
Epoch 4/20
48000/48000 [==============================] - 6s 124us/sample - loss: 0.3171 - acc: 0.8848 - val_loss: 0.3518 - val_acc: 0.8722
Epoch 5/20
48000/48000 [==============================] - 6s 129us/sample - loss: 0.2966 - acc: 0.8903 - val_loss: 0.3202 - val_acc: 0.8827
Epoch 6/20
48000/48000 [==============================] - 6s 125us/sample - loss: 0.2817 - acc: 0.8953 - val_loss: 0.3122 - val_acc: 0.8890
Epoch 7/20
48000/48000 [==============================] - 6s 127us/sample - los

In [28]:
for i in range(len(test_preds)):
  print("{}th Model: loss={} accuracy={}".format(i,log_loss(test_labels,test_preds[i]),accuracy_score(test_labels,np.argmax(test_preds[i],axis=1))))

kfold_test_preds = np.average(np.array(test_preds),axis=0)
print("KFold Average Model: loss={} accuracy={}".format(log_loss(test_labels,kfold_test_preds),accuracy_score(test_labels,np.argmax(kfold_test_preds,axis=1))))

0th Model: loss=0.3657217438034511 accuracy=0.8861
1th Model: loss=0.3571766391156306 accuracy=0.8841
2th Model: loss=0.3653581034124332 accuracy=0.8863
3th Model: loss=0.35969854553753167 accuracy=0.8863
4th Model: loss=0.38154356260789224 accuracy=0.8853
KFold Average Model: loss=0.2804664349198927 accuracy=0.9036
